In [1]:
# 역사
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv

# chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe'
# options = webdriver.ChromeOptions()
# options.add_argument('window-size=1920x1080')

# driver = webdriver.Chrome(chromedriver, options=options)

chromedriver = 'C:/dev_python/Webdriver/chromedriver.exe'
headless_options = webdriver.ChromeOptions()
headless_options.add_argument('headless')
headless_options.add_argument('window-size=1920x1080')
headless_options.add_argument('disable-gpu')
agent = "User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131"
headless_options.add_argument(agent)
headless_options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(chromedriver, options=headless_options)

areacode_arr = [1, 2, 3, 4, 5, 6, 7, 8, 31, 32, 33, 34, 35, 36, 37, 38, 39]


for areacode in areacode_arr:
    driver.get(f"https://korean.visitkorea.or.kr/list/ms_list.do?areacode={areacode}")

    print(driver.title)
    print(driver.current_url)

    tag = driver.find_element_by_css_selector("#taglist> li:nth-child(9) > button > span")

    actions = webdriver.ActionChains(driver)
    actions.move_to_element(tag)
    actions.double_click(tag)
    actions.perform()
    
    time.sleep(30)
    
    rank = driver.find_element_by_css_selector(".btn_txt > button[id='3']")
    actions = webdriver.ActionChains(driver)
    actions.move_to_element(rank)
    actions.double_click(rank)
    actions.perform()
    
    time.sleep(20)
    
    if ',' in driver.find_element_by_css_selector("#totalCnt").text:
        data_num = int(''.join(driver.find_element_by_css_selector("#totalCnt").text.split(',')))
    else:
        data_num = int(driver.find_element_by_css_selector("#totalCnt").text)
    
    print(data_num)

    travel_data = []
    
    if data_num % 10 == 0:
        length = data_num // 10
    else:
        length = (data_num // 10) + 1
    
    for j in range(length):
        if driver.find_element_by_css_selector(f".page_box > a[id='{j + 1}']"):
            page = driver.find_element_by_css_selector(f".page_box > a[id='{j + 1}']")
        actions = webdriver.ActionChains(driver)
        actions.move_to_element(page)
        actions.double_click(page)
        actions.perform()

        time.sleep(3)

        names = driver.find_elements_by_css_selector(".bdr_nor > .area_txt > .tit > a")
        regions = driver.find_elements_by_css_selector("#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li > div.area_txt > p:nth-child(2)")
        tels = driver.find_elements_by_css_selector("#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li > div.area_txt > p:nth-child(3)")
        keywords = driver.find_elements_by_css_selector("#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li > div.area_txt > p.tag")

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        photos = soup.select('#contents > div.wrap_contView.clfix > div.box_leftType1 > ul > li > div.photo > a > img')

        for i in range(len(names)):
            name = names[i].text
        #     for name in names:
        #         print(name.text)

        #     for region in regions:

            if "-" in regions[i].text:
                region = regions[i].text.split("-")[1]
                region = region[1:]
            else:
                region = regions[i].text

        #     for keyword in keywords:
            tel = tels[i].text 
            keyword = keywords[i].text.split("#")[1:]

            photo = photos[i].get('src')

            travel_dict = {
                'name' : name,
                'region' : region,
                'tel' : tel,
                'keyword' : keyword,
                'photo' : photo
            }

        #     print(travel_dict)
            travel_data.append(travel_dict)

    with open(f'./travel_data_history/{region[:2]}_history_data.csv', 'w', encoding='utf-8-sig', newline='') as file:
        fieldnames = ['name', 'region', 'tel', 'keyword', 'photo']
        csvfile = csv.DictWriter(file, fieldnames = fieldnames)
        csvfile.writeheader()
        for f in travel_data:
            csvfile.writerow(f)

대한민국구석구석 | 여행지 목록
https://korean.visitkorea.or.kr/list/ms_list.do?areacode=38
377
대한민국구석구석 | 여행지 목록
https://korean.visitkorea.or.kr/list/ms_list.do?areacode=39
48
